## Figures notebook

In [29]:
import geopandas as gpd
from mapreader.utils.load_frames import load_from_csv, load_from_geojson
from shapely import Polygon

import matplotlib.pyplot as plt
import folium
import xyzservices as xyz

### Load 100 meter patches (classification)

In [30]:
patches = load_from_csv("../post_processed_railspace_predictions_patch_df.csv")

In [31]:
patches.to_file("../post_processed_railspace_predictions_patch_df.geojson")

### Load 1000 pixel patches (text)

In [32]:
patches_1000pixel = load_from_csv("../1000pixel_patch_df.csv")

In [33]:
patches_1000pixel.to_file("../1000pixel_patch_df.geojson")

### Set up tiles

In [34]:
tiles = xyz.TileProvider(
    name="OS 2nd Edition - 6 inch",
    url="https://api.maptiler.com/tiles/uk-osgb1888/{z}/{x}/{y}?key=5f6FYax2HhTa0Z9RfXsp",
    attribution="NLS",
)

### Plot

In [35]:
# m = patches.explore(color="red", style_kwds={"weight":1}, tiles=tiles) # takes a long time due to the large number of patches

In [36]:
# m.save("100meter_patches_map.html")

In [37]:
len(patches)

586275

In [ ]:
patch = patches[patches["parent_id"]=="map_96805026.png"] # choose a single parent
patch_x, patch_y = [patch.centroid.x[0], patch.centroid.y[0]] # get centroid

m = patch[patch["new_predicted_label"]=="railspace"].explore(color="red", style_kwds={"weight":1, "fill": False}, tiles=tiles) # plot patch boundary

m

In [45]:
patch.to_file("patches.geojson") # for QGIS

In [46]:
m.save("100meter_patches_map.html")

In [47]:
patch_railspace = patch[patch["new_predicted_label"]=="railspace"] # get only railspace patches

### Load text predictions (polygons)

In [14]:
text = load_from_geojson("../geo_predictions_deduplicated.geojson")

In [15]:
text_filtered = text.drop_duplicates(subset=['geometry','text']).reset_index(drop=True)

In [16]:
text_filtered["polygon"] = text_filtered["geometry"]
text_filtered["point"] = text_filtered["geometry"].centroid

In [17]:
text_filtered_polygon = text_filtered.set_geometry("polygon", drop=True)
text_filtered_point = text_filtered.set_geometry("point", drop=True)

In [18]:
len(text_filtered)

253581

### Plot

In [19]:
text_railspace = text_filtered[text_filtered.geometry.intersects(patch_railspace.unary_union)] # get only text within the railspace patches

In [20]:
text_railspace.drop(columns=["point", "polygon"]).to_file("text_railspace.geojson")

In [ ]:
m = patch_railspace.explore(style_kwds={"weight":2, "fill": False}, color="red", tiles=tiles, cmap="viridis") # plot patch boundary

text_railspace.explore(tiles=tiles, color="blue", size=5, m=m, style_kwds={"fill":False}) # plot text polygons


In [22]:
m.save("text_on_railspace.html")

In [23]:
patch_1000pixel = patches_1000pixel[patches_1000pixel["parent_id"]=="map_96805026.png"][8:9] # choose a single patch
patch_1000pixel_x, patch_1000pixel_y = [patch_1000pixel.centroid.x[0], patch_1000pixel.centroid.y[0]] # get centroid

In [24]:
patch_1000pixel.to_file("patch_1000pixel.geojson")

In [25]:
patch_text_polygon = text_filtered_polygon[text_filtered_polygon.geometry.intersects(patch_1000pixel.geometry[0])] # get text within patch
patch_text_point = text_filtered_point[text_filtered_point.index.isin(patch_text_polygon.index)] # get text within patch

In [26]:
patch_text_polygon.drop(columns="point").to_file("patch_text_polygon.geojson")
patch_text_point.drop(columns="polygon").to_file("patch_text_point.geojson")

In [ ]:
m = patch_1000pixel.explore(color="red", style_kwds={"weight":1, "fill":False, "weight":4}, tiles=tiles) # plot patch boundary

patch_text_polygon.explore(tiles=tiles, color="blue", size=5, m=m, style_kwds={"fill":False}) # plot text polygons
patch_text_point.explore(tiles=tiles, color="blue", size=1, m=m) # plot text centroids

In [28]:
m.save("1000pixel_text_map.html")